## Building the Corpus

# Data collection

## Scrap data

Install packages

In [97]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [98]:
# Create a list containing all of the sections of react.dev/learn/describing-the-ui
section_urls = ["https://react.dev/learn/your-first-component",
                "https://react.dev/learn/importing-and-exporting-components",
                "https://react.dev/learn/writing-markup-with-jsx",
                "https://react.dev/learn/javascript-in-jsx-with-curly-braces",
                "https://react.dev/learn/passing-props-to-a-component",
                "https://react.dev/learn/conditional-rendering",
                "https://react.dev/learn/rendering-lists",
                "https://react.dev/learn/keeping-components-pure",
                "https://react.dev/learn/understanding-your-ui-as-a-tree"]

In [99]:
# Define the column names
columns = ["_id", "subsectionTitle", "url", "content", "codeSnippets", "relatedLinks","metadata"]
# Creating a dataframe that stores all of the content
df = pd.DataFrame(columns = columns)

### Appending urls and subheadings

In [100]:
# Function to scrape all of the subheadings of a page and return a list of all of the subheadings
def store_subheadings(url):
    # Fetch and parse the HTML content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find_all subheadings <h2>
    h2s = soup.find_all("h2", class_ = "mdx-heading text-3xl font-display leading-10 text-primary dark:text-primary-dark font-bold my-6")

    # Initiliaze list of h2 text
    cleaned_h2 = []
    for h2 in h2s:
        cleaned_h2.append(h2.text)
    
    return cleaned_h2

In [133]:
# Append urls and subheadings to df
def append_to_df(url_list, appended_df):
    for i in range(len(url_list)):
        subheadings = store_subheadings(url_list[i])
        # create a temporary df from the subheadings list 
        temp_df = pd.DataFrame({
            'subsectionTitle': subheadings,
            'url': [url_list[i]] * len(subheadings)
        })
        # Concat temp_df with original df
        appended_df = pd.concat([appended_df, temp_df], ignore_index=True)
    return appended_df

In [134]:
df1 = df.copy()
df1 = (append_to_df(section_urls, df1))


In [162]:
# def retrieve_content(url, df):
#     # Fetch and parse the HTML content
#     response = requests.get(url)
#     soup = BeautifulSoup(response.content, 'html.parser')

#     # Find all h2 tags
#     h2s = soup.find_all("h2", class_="mdx-heading text-3xl font-display leading-10 text-primary dark:text-primary-dark font-bold my-6")
#     for h2 in h2s:
#         # Initialize an empty list to hold the text of p tags following the current h2
#         p_texts = []
#         # Find the next sibling of h2 that is a p tag
#         next_p = h2.find_next_sibling("p")
#         while next_p:
#             p_texts.append(next_p.text)
#             next_p = next_p.find_next_sibling("p")  # Continue to find the next p tag

#         # Create a temporary df from the collected p texts
#         temp_df = pd.DataFrame({
#             'content': p_texts
#         })
#         # Concat temp_df with df
#         df = pd.concat([df, temp_df], ignore_index=True)
#     return df


In [170]:
def retrieve_content(url, current_df):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    h2s = soup.find_all("h2", class_="mdx-heading text-3xl font-display leading-10 text-primary dark:text-primary-dark font-bold my-6")
    
    for h2 in h2s:
        # Find the next siblings that are <p> tags
        p_tags = h2.find_next_siblings("p", class_="whitespace-pre-wrap my-4")
        
        # Extract the text from each <p> tag
        content = ' '.join([p.text for p in p_tags])
        
        # Logic to match and update the current_df with new content
        # This step depends on your DataFrame structure and how you identify which row to update
        # Example: Assume we match by the h2 text (assuming it's unique and exists as a column in current_df)
        h2_text = h2.text
        current_df.loc[current_df['subsectionTitle'] == h2_text, 'content'] = content
    
    return current_df

In [171]:
df2 = df1.copy()
for url in section_urls:
    df2 = retrieve_content(url, df1)

In [176]:
df2.head()

,_id,subsectionTitle,url,content,codeSnippets,relatedLinks,metadata
0,NaN,Components: UI building blocks,https://react.dev/learn/your-first-component,"On the Web, HTML lets us create rich structure...",NaN,NaN,NaN
1,NaN,Defining a component,https://react.dev/learn/your-first-component,"Traditionally when creating web pages, web dev...",NaN,NaN,NaN
2,NaN,Using a component,https://react.dev/learn/your-first-component,Now that you’ve defined your Profile component...,NaN,NaN,NaN
3,NaN,Recap,https://react.dev/learn/your-first-component,,NaN,NaN,NaN
4,NaN,The root component file,https://react.dev/learn/importing-and-exportin...,"In Your First Component, you made a Profile co...",NaN,NaN,NaN
